In [2]:
# !kaggle competitions download -c ds2-predictive-modeling-challenge

100%|██████████████████████████████████████| 3.81M/3.81M [00:01<00:00, 3.42MB/s]

  0%|                                                | 0.00/211k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 211k/211k [00:00<00:00, 3.31MB/s]
100%|████████████████████████████████████████| 948k/948k [00:00<00:00, 2.90MB/s]

  0%|                                                | 0.00/236k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 236k/236k [00:00<00:00, 3.83MB/s]


In [3]:
# !unzip train_features.csv.zip

Archive:  train_features.csv.zip
  inflating: train_features.csv      


In [4]:
# !unzip test_features.csv.zip
# !unzip train_labels.csv.zip

Archive:  test_features.csv.zip
  inflating: test_features.csv       
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
import warnings
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression,f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
# import xgboost as xgb

In [41]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train_labels = pd.read_csv('train_labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

train_features.shape, test_features.shape, train_labels.shape, sample_submission.shape

((59400, 40), (14358, 40), (59400, 2), (14358, 2))

In [42]:
pd.options.display.max_columns = 100
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [43]:
train_features.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [44]:
train_features.describe(exclude=np.number)

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [45]:
train = train_features.copy()

In [46]:
train_features.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [47]:
year_mean = train[train['construction_year']>0]['construction_year'].mean()
year_mean = round(year_mean)

test_year_mean = test_features[test_features['construction_year']>0]['construction_year'].mean()
test_year_mean = round(test_year_mean)


train.loc[train['construction_year']==0, 'construction_year'] = int(year_mean)
test_features.loc[test_features['construction_year']==0,'construction_year'] = int(test_year_mean)

In [48]:
import random

def random_std(year):
    
    std = year - random.randint(0,10)
    
    return std    

def random_tsh(amount):
    
    std = amount + np.random.uniform(1062.35, 2957.82)
    return std

In [49]:
train_tsh_mean = train[train['amount_tsh']>0]['amount_tsh'].mean()
test_tsh_mean = test_features[test_features['amount_tsh']>0]['amount_tsh'].mean()

In [50]:
train.loc[train['construction_year']==1997, 'construction_year'].apply(random_std)
test_features.loc[test_features['construction_year']==1997, 'construction_year'].apply(random_std)
train.loc[train['amount_tsh']==0, 'amount_tsh'].apply(random_tsh)
test_features.loc[test_features['amount_tsh']==0, 'amount_tsh'].apply(random_tsh)

train.shape, test_features.shape

((59400, 40), (14358, 40))

In [51]:
train['funder'].fillna('?',inplace=True)
train['installer'].fillna('?',inplace=True)
train['subvillage'].fillna('?',inplace=True)
train['public_meeting'].fillna('?',inplace=True)
train['scheme_management'].fillna('?',inplace=True)
train['scheme_name'].fillna('?',inplace=True)
train['permit'].fillna('?',inplace=True)
test_features['funder'].fillna('?',inplace=True)
test_features['installer'].fillna('?',inplace=True)
test_features['subvillage'].fillna('?',inplace=True)
test_features['public_meeting'].fillna('?',inplace=True)
test_features['scheme_management'].fillna('?',inplace=True)
test_features['scheme_name'].fillna('?',inplace=True)
test_features['permit'].fillna('?',inplace=True)

In [52]:
train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [53]:
train['age'] = (2019 - train['construction_year']).astype(int)
test_features['age'] = (2019 - test_features['construction_year']).astype(int)

In [54]:
drop_these=[
    'date_recorded',
    'wpt_name',
    'recorded_by',
    'lga',
    'ward',
    'scheme_name',
    'funder',
    'installer',
    'num_private',
    'subvillage',
    'basin',
    'region_code',
    'district_code',
    'longitude',
    'latitude',
    'region'
]
train.drop(columns=drop_these,inplace=True)
test_features.drop(columns=drop_these,inplace=True)

train.shape, test_features.shape

((59400, 25), (14358, 25))

In [55]:
ohe = ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',impute_missing=False)

train_features = ohe.fit_transform(train)
test_features = ohe.fit_transform(test_features)

train_features.shape, test_features.shape

((59400, 151), (14358, 149))

In [56]:
columns = train_features.columns
test_cols = test_features.columns
for col in columns:
    if col not in test_cols:
        print(col)

scheme_management_None
extraction_type_other - mkulima/shinyanga


In [57]:
train_features.drop(columns=['scheme_management_None','extraction_type_other - mkulima/shinyanga'],inplace=True)
train_features.columns == test_features.columns

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False,  True, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False, False,  True,  True, False,  True, False,  True,  True,
       False,  True, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False,

In [58]:
len(train_features.columns) == len(test_features.columns)

True

In [59]:
y_train = train_labels['status_group']
y_train.shape

(59400,)

In [28]:
model = LogisticRegression()
model.fit(train_features, y_train)
model.coef_

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([[ 4.61737865e-01, -2.22491153e-01,  1.12581132e-02,
        -1.62960460e-01,  3.23087545e-02,  2.01721553e-01,
         1.67559864e-01,  3.28073888e-02, -4.93064585e-02,
         5.10662415e-02, -2.58916152e-01, -1.71576555e-01,
         2.08683927e-01, -2.87703461e-01, -1.79902908e-01,
         7.94367687e-02,  1.84361100e-01, -1.57550769e-01,
         7.46081672e-02, -2.88470062e-02,  1.36886302e-02,
         1.92032051e-01,  4.99172039e-02, -2.41965805e-01,
         2.72518573e-02,  7.44494944e-02,  9.69549541e-02,
         9.15036686e-02, -1.03059002e-01,  2.10438671e-01,
         8.25049097e-02, -2.48659084e-01, -2.12045997e-01,
        -1.02662422e-02,  7.47775252e-03, -1.70429967e-02,
        -1.84577700e-01,  3.66719639e-02,  1.47889186e-01,
         5.85550245e-02,  9.03723770e-02, -2.55441669e-02,
         2.39461190e-01,  3.11955092e-02, -2.29950156e-01,
        -1.50568310e-01, -4.98470155e-02, -5.36712318e-03,
         1.13512606e-01,  1.45257607e-02, -2.25008311e-0

In [21]:
# pipeline = make_pipeline(
#     RobustScaler()
#     DecisionTreeClassifier()
# )
# param_grid={
#     'decisiontreeclassifier__criterion': ['gini','entropy'] 
#     'decisiontreeclassifier__max_features': [40,80,100,182]
#     'decisiontreeclassifier__max_depth': [25,50,100] 
# }

# gridsearch = GridSearchCV(pipeline,paramgrid=paramgrid,cv=5,scoring='neg_mean_absolute_error',
#                           verbose=1,return_train_score=True)

# gridsearch.fit(train_features, y_train)
# run this overnight

In [29]:
scaler = StandardScaler()
model = DecisionTreeClassifier()
scaler.fit(train_features)
X_train = scaler.transform(train_features)
X_test = scaler.transform(test_features)

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [61]:
model.set_params(max_depth = 50, max_features=20, criterion='gini',min_samples_leaf=3)
model.fit(X_train,y_train)

submission=sample_submission.copy()
submission['status_group'] = model.predict(X_test)
submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [62]:
submission.to_csv('submission-009.csv',index=False)

In [35]:
model = LogisticRegression()
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(train_features)
X_test = scaler.transform(test_features)
model.set_params(solver='lbfgs', max_iter=1000)
model.fit(X_train,y_train)

submission=sample_submission.copy()
submission['status_group'] = model.predict(X_test)
submission.head()

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [36]:
submission.to_csv('submission-007.csv',index=False)

In [63]:
scaler = RobustScaler()
model = RandomForestClassifier()
scaler.fit(train_features)
X_train = scaler.transform(train_features)
X_test = scaler.transform(test_features)


In [64]:
model.set_params(n_estimators=200, min_samples_leaf=4, n_jobs=-1)
model.fit(X_train,y_train)


submission=sample_submission.copy()
submission['status_group'] = model.predict(X_test)
submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [65]:
submission.to_csv('submission-010.csv',index=False)

In [66]:
# submission9 = pd.read_csv('submission-009.csv')
# submission10 = pd.read_csv('submission-010.csv')